## The EDA of Physionet Data set regarding "A Large Scale 12 Lead Electrocardiogram Database for Arrhythmia Study 1.0.0"

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import os
from scipy.signal import butter, filtfilt
import biosppy.signals.ecg as ecg
import psycopg2
from sklearn.decomposition import IncrementalPCA

### Declaring Global variables

In [9]:
dataset_path = r'..\a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0\WFDBRecords\\'
count = 0
# Adjusted cutoff frequencies for normalized signal values
lowcut = 0.66  # Lower cutoff frequency in Hz
highcut = 46.0   # Higher cutoff frequency in Hz
index = 0

### Visual each ECG Signal from the provided dataset using wfdb function

In [10]:
def visualize_data(record,name):
    _,axes = plt.subplots(12,1,figsize=(12,20))
    
    
    leads = ["I", "II", "III", "avR", "avF", "aVL", "V1", "V2", "V3", "V4", "V5", "V6"]
    
    for i, lead in enumerate(leads):
        axes[i].plot(record[:, i])
        axes[i].set_ylabel(lead)
        
    plt.legend(leads)
    plt.savefig(f'{name}{count}.png')
    plt.show()

### Filter for noise removal and signal normalization

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist_freq = 0.5 * fs
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b, a = butter(order, [low, high], btype='band', analog=False)
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    filtered_data = filtfilt(b, a, data)
    return filtered_data


### Calculating Heart rate

In [ ]:
def calculate_heartrate(record,fs):
    ecg_signal = record[:,index]
    # Process ECG signal to detect R-peaks
    rpeaks = ecg.engzee_segmenter(ecg_signal, sampling_rate=500)[0]
    # Calculate RR intervals (in seconds)
    rr_intervals = np.diff(rpeaks) / fs  # Sampling rate is 500 Hz
    # Calculate HRV metrics
    # sdnn = np.std(rr_intervals)  # Standard deviation of NN intervals
    # rmssd = np.sqrt(np.mean(np.diff(rr_intervals) ** 2))  # Root mean square of successive differences
    heart_rate = 60 * len(rr_intervals) / np.sum(rr_intervals)

    # Print HRV metrics
    print("Heart rate:", heart_rate)
    # Print HRV metrics
    # print("SDNN:", sdnn)
    # print("RMSSD:", rmssd)

### Calculating QRS Interval

In [ ]:
def calculate_qrs_interval(record,fs):
    ecg_signal = record[:,index]
    # Process ECG signal to detect QRS complexes
    qrs_indices = ecg.hamilton_segmenter(ecg_signal, sampling_rate=500)[0]

    # Measure QRS duration for each QRS complex
    qrs_durations = []
    for qrs_index in qrs_indices:
        # Define QRS complex boundaries (e.g., ±50 ms around QRS peak)
        qrs_start = max(0, qrs_index - 0.05 * fs)  # 50 ms before QRS peak
        qrs_end = min(len(ecg_signal), qrs_index + 0.05 * fs)  # 50 ms after QRS peak
        
        # Calculate QRS duration (in seconds)
        qrs_duration = (qrs_end - qrs_start) / float(fs)  # Sampling rate is 500 Hz
        qrs_durations.append(qrs_duration)

    # Calculate average QRS duration
    avg_qrs_duration = np.mean(qrs_durations)# Format average QRS duration to display up to 3 decimal places
    formatted_avg_qrs_duration = "{:.3f}".format(avg_qrs_duration)

    print("Average QRS Duration:", formatted_avg_qrs_duration, "seconds")

### Applying Partial Principle Component Analysis

In [7]:
def visualize_pca_incremental(data_generator, batch_size=1000):
    # Initialize Incremental PCA
    ipca = IncrementalPCA(n_components=2)

    # Process data in batches
    for batch_data in data_generator:
        # Flatten the batch
        flattened_data = batch_data.reshape((batch_data.shape[0], -1))
        # Partial fit IPCA on the flattened batch
        ipca.partial_fit(flattened_data)

    # Get the principal components
    pcs = ipca.components_

    # Visualize the coefficients of the first two principal components
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, 60001), pcs[0], marker='o', linestyle='-')
    plt.title('Coefficients of Principal Component 1')
    plt.xlabel('Feature Index')
    plt.ylabel('Coefficient')

    plt.subplot(1, 2, 2)
    plt.plot(range(1, 60001), pcs[1], marker='o', linestyle='-')
    plt.title('Coefficients of Principal Component 2')
    plt.xlabel('Feature Index')
    plt.ylabel('Coefficient')

    plt.tight_layout()
    plt.show()

    # Print the explained variance ratio
    print("Explained Variance Ratio:")
    print(ipca.explained_variance_ratio_)


In [8]:
def data_generator(batch_size=1000):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="EDA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()

    try:
        # Fetch data from the database in batches
        offset = 0
        while True:
            # Fetch up to batch_size entries from the database
            cursor.execute(f"SELECT p_signal FROM ECG LIMIT {batch_size} OFFSET {offset}")
            rows = cursor.fetchall()
            
            if not rows:
                break  # No more data to fetch
            
            # Convert fetched data to numpy array
            batch_data = np.array([row[0] for row in rows])
            
            
            
            
            
            yield batch_data

            # Move to the next batch
            offset += batch_size

    finally:
        # Close database connection
        cursor.close()
        conn.close()

### Data Inserted into database from the data folder downloaded from physionet

In [ ]:
def data_Insertor():
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="EDA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    
    insertQuery = '''INSERT INTO ECG (ID, AGE, GENDER,DX,p_signal) VALUES (%s,%s,%s,%s,%s)'''
    try:
        createQuery = '''CREATE TABLE IF NOT EXISTS ECG(
                ID varchar(10) primary key,
                AGE int,
                GENDER varchar(10),
                DX TEXT,
                p_signal BYTEA)'''
        cursor.execute(createQuery)
            
        for dir1 in os.listdir(dataset_path):
            dir1_path = os.path.join(dataset_path, dir1)
            
            # Loop through the subdirectories
            for dir2 in os.listdir(dir1_path):
                dir2_path = os.path.join(dir1_path, dir2)

                # Loop through the ECG records
                for file_name in os.listdir(dir2_path):
                    if file_name.endswith('.mat'):
                        file_path = os.path.join(dir2_path, file_name[:-4])
                        record = wfdb.rdrecord(file_path)        
                        try:
                            age = int(record.comments[0][5:])
                        except:
                            age = 62
                            
                        insertValues = (record.record_name,age,record.comments[1][5:],record.comments[2][4:],psycopg2.Binary(record.p_signal.tobytes()))
                        cursor.execute(insertQuery,insertValues)
                        conn.commit()
                        print(record.record_name)
            
    finally:
        # Close database connection
        cursor.close()
        conn.close()

data_Insertor()

In [ ]:
# def data_Check():
#     # Connect to the PostgreSQL database
#     conn = psycopg2.connect(
#         dbname="EDA",
#         user="postgres",
#         password="admin",
#         host="localhost",
#         port="5432"
#     )
#     cursor = conn.cursor()
    
#     selectQuery = '''select p_signal from ecg where id = %s'''
#     queryValues = ('JS00001',)
#     try:
#         cursor.execute(selectQuery,queryValues)
#         array_binary = cursor.fetchone()[0]
#         array = np.frombuffer(array_binary, dtype=float)  # Specify the dtype of the array
#         array = array.reshape((5000,12)) 
#         conn.commit()
#         print(array.shape)
#         print(array)
#     finally:
        
#         cursor.close()
#         conn.close()
        
# data_Check()

In [ ]:


batch_signal = []
record = None
# Initialize an empty DataFrame to hold the final result
final_df = None
# Loop through the first-level directories
for dir1 in os.listdir(dataset_path):
    dir1_path = os.path.join(dataset_path, dir1)
    
    # Loop through the subdirectories
    for dir2 in os.listdir(dir1_path):
        dir2_path = os.path.join(dir1_path, dir2)

        # Loop through the ECG records
        for file_name in os.listdir(dir2_path):
            if file_name.endswith('.mat'):
                file_path = os.path.join(dir2_path, file_name[:-4])
                count = count + 1
                if count < 10000:
                    record = wfdb.rdrecord(file_path)
                    # print(record.__dict__)
                    # visualize_data(record.p_signal, "plot")
                    # Initialize filtered_p_signal array
                    filtered_p_signal = np.zeros_like(record.p_signal)

                    # Apply band-pass filter to each lead separately
                    for lead in range(12):
                        filtered_signal = bandpass_filter(record.p_signal[:, lead], lowcut, highcut, record.fs)
                        filtered_p_signal[:, lead] = filtered_signal
                    
                    batch_signal.append(filtered_p_signal)
                    print(np.array(batch_signal).shape)
                    if len(batch_signal) == 1000:
                        visualize_pca_incremental(np.array(batch_signal))
                        batch_signal = []
                    
                    # visualize_pca(filtered_p_signal)
                    # visualize_data(filtered_p_signal, "Normailzed")
                    # calculate_heartrate(filtered_p_signal,record.fs)
                    # calculate_qrs_interval(filtered_p_signal,record.fs)
                else:
                    
                    break
        # # Optionally, save the final DataFrame to a CSV file
        # final_df.to_csv()
        # final_df.drop(final_df.index, inplace=True)
        # final_df=None




## The Block to have commented out code:

# _, singular_values, _ = np.linalg.svd(record.p_signal)
    # print(np.count_nonzero(singular_values))
    # print(record.p_signal.T.shape)
    # print(np.linalg.matrix_rank(record.p_signal.T))
    
    # max(rank) = min(row, coulmns)
    
    # _, singular_values, _ = np.linalg.svd(record.p_signal.T)
    # print(np.count_nonzero(singular_values))
    
    
    
    #Printing the data set attribute p_signal to get the idea of the signal of ecg
    # print(record.record_name)
    # print(record.p_signal.shape)
    # print(type(record.p_signal))
    # print(np.sum(np.isnan(record.p_signal) | (record.p_signal == None)))
    # print(record.__dict__)
    # wfdb.plot_wfdb(record)
# visualize_ecg(dataset_path) 
